### LLMs with personalities

In [68]:
# imports
import ollama
from dotenv import load_dotenv
import openai
from google import genai
import pandas as pd

In [81]:
load_dotenv(dotenv_path="../../.env")

True

### Celebrities: Elon Musk vs Gordon Ramsay

In [69]:
# using BeautifulSoup, we will extract data from both celebrities from X.
from bs4 import BeautifulSoup
import requests






class WebScrapper:
    def __init__(self):
        self.headers = {"User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/58.0.3029.110 Safari/537.3"}
        self.url = "https://www.personalityclub.com/personality-types/"
        self.contents, self.titles, self.personalities = self.get_content()
    
    def get_urls(self):
        response = self.get_response("https://www.personalityclub.com/personality-types/")
        soup = BeautifulSoup(response.text, "html.parser")
        urls = []
        divs = soup.find_all("div", {"class":"one-fourth first"})
        for div in divs:
            h3_a = div.find("h3").a  # find the <a> inside <h3>
            if h3_a and h3_a.get("href"):
                url = h3_a["href"]
        divs = soup.find_all("div", {"class":"one-fourth"})
        if divs:
            for div in divs:
                h3_a = div.find("h3").a  # find the <a> inside <h3>
                if h3_a and h3_a.get("href"):
                    url = h3_a["href"]
                    urls.append(url)
        return urls


    def get_response(self, url):
        response = requests.get(headers=self.headers, url=url)
        return response
    
    def get_content(self):
        """This function will scrape the content from the website.
            Returns:
                self: WebScrapper
                (self.personality): dict
                (self.title): str
                (self.content): str
        """
        urls = self.get_urls()
        contents = []
        titles = []
        personalities = []
        for url in urls:
            response = self.get_response(url=url)
            soup = BeautifulSoup(response.text, "html.parser")
            title = soup.find("h1", {"class":"entry-title"})
            content = soup.find("div", {"class":"entry-content"})
            personality = ""
            for paragraph in content.find_all("p"):
            # If this paragraph contains "Keirsey Temperament:", stop BEFORE adding it
                if "Keirsey Temperament:" in paragraph.get_text():
                    break
                personality += paragraph.get_text(strip=True) + "\n"
            titles.append(title.text)
            contents.append(content.text)
            personalities.append(personality)
        return contents, titles, personalities


webscrapper = WebScrapper()
webscrapper.personalities



['Motto:Organizing everything (Source)\nDescription:(Source)\nISTJs thrive on organization. They keep their lives and environments well-regulated. They bring painstaking attention to detail in their work and will not rest until a job is well completed.\nThey are often dissatisfied with unresolved issues, whether in life or in fiction.\n\nISTJs are faithful, logical, organized, sensible, and earnest traditionalists. They earn success by thoroughness and dependability.\nShutting out distractions, they take a practical, logical approach to their endeavors. Realistic and responsible, they work steadily toward their goals.\nThey enjoy creating order in both their professional and personal lives.\nDespite their focus on their internal world, ISTJs prefer dealing with the present and the factual. Keen observers of life, they weigh various options when making decisions.\nISTJs are well-prepared for most eventualities and have a good understanding of most situations. They believe in practical o

In [65]:
import pandas as pd
pd.DataFrame({"titles": webscrapper.titles, "personalities": webscrapper.personalities}).to_csv("personalities.csv", index=False)

### GenAI


In [80]:
# configure models
OLLAMA_MODEL = "qwen:7b"
OPEN_AI_MODEL = "gpt-4o-mini"
GOOGLE_MODEL = "gemini-2.0-flash"

### With Ollama


In [ ]:
df = pd.read_csv("personalities.csv")
system_prompt = f"""You are a helpful assistant that helps people find their personality type based on their preferences and behaviors. 
                    Based on multiple questions and answers, you will tell the user what personality they most resemble." \
                    Given this DataFrame: {df}, I want you to based on the answers to the questions, determine which personality type the user most closely resembles.
                    """
messages = [{"role":"system", "content": system_prompt}]

user_questions = [
    "Do you prefer to spend time alone or with others?",
    "Are you more of a thinker or a feeler?",
    "Do you like to plan things out or be spontaneous?",
    "Are you more of a detail-oriented person or a big-picture thinker?",
    "Do you prefer to follow rules or go with the flow?"]
for question in user_questions:
    messages.append({"role":"assistant", "content": question})
    messages.append({"role":"user", "content":input(question)})

completion = ollama.chat(
    model="qwen:7b",
    messages=messages,
)

answer = completion.message.content
print(answer)

Based on your answers, it seems like you might be an INFJ (The Idealist). INFJs are known for their big-picture thinking, spontaneity, ability to see beyond rules, and a strong desire to make a positive impact on others. However, everyone's personality is complex and influenced by many factors, so this is just one possible match.


### With OpenAI

In [83]:
messages = [{"role":"system", "content": system_prompt}]

for question in user_questions:
    messages.append({"role":"assistant", "content": question})
    messages.append({"role":"user", "content":input(question)})

completion = openai.chat.completions.create(
    model=OPEN_AI_MODEL,
    messages=messages,
)

answer = completion.choices[0].message.content
print(answer)

Based on your responses, you seem to closely resemble the **INTP – THE SCIENTIST** personality type. This type is characterized by:

- A preference for being alone (Introverted)
- A thinking approach to situations (Thinking)
- A spontaneous nature (Perceiving)
- A focus on big-picture ideas rather than minute details (Intuitive)

INTPs are analytical, innovative, and often enjoy exploring concepts and theories. They tend to go with the flow in their thinking and decision-making.


# Generate the UI with Gradio

In [88]:
print(messages)

[{'role': 'system', 'content': 'You are a helpful assistant that helps people find their personality type based on their preferences and behaviors. \n                    Based on multiple questions and answers, you will tell the user what personality they most resemble."                     Given this DataFrame:                    titles                                      personalities\n0    ISTJ – THE INSPECTOR  Motto:Organizing everything (Source)\\nDescript...\n1     ISFJ – THE NURTURER  Motto:Meeting obligations (Source)\\nDescriptio...\n2    INFJ – THE COUNSELOR  Motto:One cause after another (Source)\\nDescri...\n3   INTJ – THE MASTERMIND  Motto:Can improve on anything (Source)\\nDescri...\n4    ISTP – THE CRAFTSMAN  Motto:Working with facts and hands (Source)\\nD...\n5       ISFP – THE ARTIST  Motto:Marching to a different drummer (Source)...\n6     INFP – THE IDEALIST  Motto:Pursuing ideals (Source)\\nDescription:(S...\n7    INTP – THE SCIENTIST  Motto:Always analytical (Sour